# モデル

Transformerの下位レベルAPIを見る - Transformer自体にPytorchコードをラップするモデル。

このノートブックは、低コストまたは無料のT4ランタイムで実行できます。

## ご注意ください

このラボの中央にいくつかの新しい素材を追加して、Transformerが実際に何であるかについてより多くの直感を得ました。コースの後半では、LLMをファインチューニングすると、これをより深く理解できます。

## リマインダー：Colabを使用するための2つの重要なプロチップ：

**Pro-tip 1:**

すべてのColabの上部には、いくつかのpipインストールがあります。これを実行すると、pipからエラーを受信する場合があります。

> gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.

これらのpip互換性エラーは安全に無視できます。そして、バージョン番号を変更することでそれらを修正しようとするのは魅力的ですが、実際に実際の問題を導入します！

**Pro-tip 2:**

Colabを実行している最中に、次のようなエラーが発生する可能性があります。

> Runtime error: CUDA is required but not available for bitsandbytes. Please consider installing [...]

これは非常に誤解を招くエラーメッセージです。パッケージのバージョンを変更しないでください。

これは、GoogleがColabランタイムを切り替えたために実際に起こります。おそらくGoogle Colabが忙しすぎたためです。解決策は次のとおりです。

1. カーネルメニュー >> ランタイムを切断して削除
2. 新鮮なメニューからColabをリロードし、メニュー >> すべての出力をクリア
3. 右上のボタンを使用して新しいT4に接続
4. 右上のメニューから「リソースを表示」を選択して、GPUがあることを確認
5. Colab内のセルを上から下へ、pipのインストールから始めて再実行

そして、すべてがうまくいくはずです - そうでなければ、私に尋ねてください！

In [1]:
# これにより、pip依存関係の競合に関する「エラー」が得られる場合は、それを無視してください！それは何にも影響しません。
!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q requests bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 109.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 98.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB ? eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# import

# Pythonのgcモジュール
import gc
# PyTorch
import torch

# Google Colab で Secret を使う
from google.colab import userdata
# Hugging Faceにログイン
from huggingface_hub import login
# Hugging Face の transformers の ライブラリ
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig

# Hugging Faceにサインイン

講義でこれについては言及していませんでしたが、以前に行ったことがない場合は、Hugging Face Hubにログインする必要がある場合があります。

1. まだ行っていない場合は、https://huggingface.coで **free** Hugging Faceアカウントを作成し、右上のユーザーメニューから[設定]に移動します。次に、新しいAPIトークンを作成し、書き込み許可を与えます。  

**重要** Hugging Face APIキーを作成する場合、[書き込み]タブをクリックして、キーの読み取り/書き込み許可を必ず選択してください。そうしないと、後で問題が発生する可能性があります。

2. ここに戻って、サイドパネルの「キー」アイコンを左側に押して、新しい秘密を追加します。  
  名前フィールドに「HF_TOKEN」を入れます  
  値フィールドに実際のトークンを置きます： `hf _...`  
  「ノートブックからのアクセス	」スイッチがオンになっていることを確認してください。

3. 下のセルを実行してログインします。各Colabでこれを行う必要があります。
  SecretをColabに入力する必要なく、秘密を管理するための本当に便利な方法です。
  また、以下の行を単純に上書きするショートカットもあります。`hf_token =" hf _.... "`  
  しかし、これはベストプラクティスではありません。
  Colabを共有しないように注意する必要があるからです。
  そして、Colabについての素晴らしいことの1つは、それらを共有できることです！

In [4]:
# Hugging Faceにログイン

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [5]:
# モデルを指示します
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"
PHI3 = "microsoft/Phi-3-mini-4k-instruct"
GEMMA2 = "google/gemma-2-2b-it"
QWEN2 = "Qwen/Qwen2-7B-Instruct" # あなたのためのエクササイズ
MIXTRAL = "mistralai/Mixtral-8x7B-Instruct-v0.1" # これがGPUメモリに合わない場合は、Hubから他を試してみてください

In [6]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Tell a light-hearted joke for a room of Data Scientists"}
  ]

# Metaのllama 3.1へのアクセス

Fantastic Llama 3.1を使用するには、Metaは利用規約に署名する必要があります。

Hugging Faceモデルの指示ページにアクセスしてください。:
https://huggingface.co/meta-llama/Meta-Llama-3.1-8B

ページの上部には、彼らの条件に同意する方法に関する指示があります。可能であれば、Hugging Faceアカウントと同じメールを使用する必要があります。

私の経験では、承認は数分後に行われます。 3.1モデルに承認されると、3.1ファミリのモデル全体に適用されます。何らかの理由で、Metaがアクセスを承認しない場合があります。それが発生した場合は、[この](https://colab.research.google.com/drive/1deJO03YZTXUwcq2vzxWbiBhrRuI29Vo8?usp=sharing)トラブルシューティングをフォローしてください。

次のセルがあなたにエラーを与えた場合は、次のことを確認してください。  
1. Hugging Faceにログインしていますか？ `login()`を実行してみてください。APIキーを設定したか確認してください。
2. 完全な読み取りおよび書き込みアクセス許可でAPIキーを設定しましたか？
3. 上記のリンクがあるLlama3.1ページにアクセスした場合、上部近くのモデルにアクセスできることがわかりますか？

また、このトラブルシューティングコラブをセットアップして、Hugging Faceの接続性の問題を診断しようとします。  
https://colab.research.google.com/drive/1deJO03YZTXUwcq2vzxWbiBhrRuI29Vo8?usp=sharing


In [7]:
# 量子化構成
# BitsAndBytesConfig という設定クラスを使って、量子化（Quantization）されたモデルをメモリにロード
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,                         # 4bit 量子化を有効にするとメモリ使用量が劇的に削減される。
    bnb_4bit_use_double_quant=True,            # 二重量子化は量子化の誤差をさらに圧縮し精度劣化を最小限に抑える
    bnb_4bit_compute_dtype=torch.bfloat16,     # 計算時の内部精度を bfloat16 に設定し誤差低減、メモリ効率向上
    bnb_4bit_quant_type="nf4"                  # 正規分布ベースの量子化形式を用いることで精度の低下が抑える
)

In [8]:
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [9]:
# Model

# 8Bクラスのモデルの safetensors ファイルは 20-22 GB 前後で、GPUメモリは 32-40 GB 前後
# 量子化してロードすると、8bit量子化 で 10-12GB、13-16GB、4bit量子化 で 5-6GB、6-8GB前後
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
memory = model.get_memory_footprint() / 1e6
print(f"Memory footprint: {memory:,.1f} MB")

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Memory footprint: 5,591.5 MB


## Transformerモデルの内部を覗いてみる

次のセルは、LlamaのHugging Face `Model`オブジェクトを印刷します。

このモデルオブジェクトは、PythonフレームワークPytorchで実装されたニューラルネットワークです。

ニューラルネットワークは、2017年にGoogle Scientistsによって発明されたTransformerアーキテクチャを使用しています

私たちは理論に深く入り込むつもりはありませんが、これはTransformerが実際に何であるかについて何らかの直観を得る機会です。

ニューラルネットワークをまったく初めて使用する場合は、[YouTube intro playlist](https://www.youtube.com/playlist?list=PLWHe-9GP9SMMdl6SLaovUQF2abiLGbMjs)をご覧ください。

次に、次のセルにプリントされるニューラルネットワークの層を見てください。

 - レイヤ`(layers)`で構成される。
 - 「埋め込み」`embed_tokens`はトークンを4,096の寸法ベクトルに変える。
 - 「デコーダ・レイヤ」`LlamaDecoderLayer`と呼ばれる32セットのレイヤがある。
 - 各デコーダ層には、3種類のレイヤが含まれる。
   - （a）自己触媒層`(self_attn)`
   - （b）多層パーセプトロン`(mlp):`層
   - （c）バッチノルム層`(input_layernorm)`
 - 最後のLMヘッド・レイヤ`(lm_head)`により、出力が生成される。

モデルが4ビットに量子化されていることに注意してください。

この時点で理論に深く入り込む必要はありませんが、あなたが望むなら、私たちの共通の友人にこのプリントを取り、各レイヤを歩くためのチュートリアルを作るように頼みました。これは、各ポイントの寸法も調べます。興味がある場合は、次のセルを実行した後、このチュートリアルを作成してください。

https://chatgpt.com/canvas/shared/680cbea6de688191a20f350a2293c76b

In [10]:
# このセルを実行して、印刷されるものを見てください。レイヤを調査します
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((409

### そして、あなたがさらに深くTransformerに行きたいなら

モデル内の各レイヤーを見ることに加えて、実際にPytorchを使用してLlamaを実装するHugging Faceコードを見ることができます。

これがHugging Face Transformersリポジトリです：  
https://github.com/huggingface/transformers

そしてこの中に、ここにllama4のコードがあります：  
https://github.com/huggingface/transformers/blob/main/src/transformers/models/llama4/modeling_llama4.py

明らかに、この詳細に入ることはまったく必要ではありません -  AIエンジニアの仕事は、PytorchでTransformersをコーディングするのではなく、LLMを選択、最適化、ファインチューニング、適用することです。 OpenAI、Meta、およびその他のフロンティア・ラボは、これらのモデルの構築とトレーニングを何百万人も費やしました。しかし、あなたが興味があるなら、それは魅力的なウサギの穴です！

In [11]:
# それでは、モデルを実行しましょう！
outputs = model.generate(inputs, max_new_tokens=80)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

Tell a light-hearted joke for a room of Data Scientists<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Why did the logistic regression model go to therapy?

Because it was struggling to classify its emotions.<|eot_id|>


In [12]:
# メモリをクリーンアップします
# Kuan L.これを適切に解放するのを手伝ってくれてありがとう！
# 右上で「リソースを表示」を選択してGPUメモリを表示すると、すぐにドロップダウンしない可能性があります
# ただし、メモリは、後のコードで新しいモデルで使用できるようです。

del model, inputs, tokenizer, outputs
gc.collect()
torch.cuda.empty_cache()

## コードの次のブロックに関するいくつかの簡単なメモ：

TextStreamerと呼ばれるHugging Faceユーティリティを使用しているので、結果が戻ってきました。

結果をストリーミングするには、単に置き換えます。  

`outputs = model.generate(inputs, max_new_tokens=80)`  

を、以下に：  

`streamer = TextStreamer(tokenizer)`  
`outputs = model.generate(inputs, max_new_tokens=80, streamer=streamer)`

また、Chatテンプレートを作成するために呼び出しに `add_generation_prompt=True` という引数を追加しました。これにより、Userプロンプトがどのように継続するかを予測するのではなく、Phi3が質問に対する回答を生成することが保証されます。これを`False`に設定して、何が起こるかを確認してみてください。この議論についてはこちらを読むことができます。

https://huggingface.co/docs/transformers/main/en/chat_templating#what-are-generation-plompts

問題を提起してくれたStudent Piotr Bに感謝します！

In [16]:
# ストリーミングと生成のプロンプトを追加した関数
def generate(model, messages, add_generation_prompt=True):
  tokenizer = AutoTokenizer.from_pretrained(model)
  tokenizer.pad_token = tokenizer.eos_token
  inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=add_generation_prompt).to("cuda")
  streamer = TextStreamer(tokenizer)
  model = AutoModelForCausalLM.from_pretrained(model, device_map="auto", quantization_config=quant_config)
  outputs = model.generate(inputs, max_new_tokens=80, streamer=streamer)
  del model, inputs, tokenizer, outputs, streamer
  gc.collect()
  torch.cuda.empty_cache()

In [17]:
# 推論時、apply_chat_templateは<|assistant|>まで生成し、generateはコレ以降を生成する。
generate(PHI3, messages, True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<|system|> You are a helpful assistant<|end|><|user|> Tell a light-hearted joke for a room of Data Scientists<|end|><|assistant|> Why did the data scientist break up with the algorithm?

Because it was always calculating the wrong variables in their relationship!<|end|>


In [18]:
# 学習時、apply_chat_templateは<|endoftext|>まで生成し、generateはコレ以降を生成する（<|user|>などや無関係な出力を始めてしまう）。
generate(PHI3, messages, False)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<|system|> You are a helpful assistant<|end|><|user|> Tell a light-hearted joke for a room of Data Scientists<|end|><|endoftext|> 
<|user|> I need a Dockerfile for a Node.js app using the Node 12 Alpine image. Set up a work directory, copy the package.json and package-lock.json, install dependencies, copy the source code, expose port 3000, and set the command to run the app. Keep it simple and efficient.<|end|>


## GoogleからGemmaにアクセス

Googleは、Gemmaを使用する前にHugging Faceで条件を受け入れることを求めていることも、学生（Alex K！）を教えてください（ありがとう、Alex K！）。

このリンクのモデルページにアクセスして、アクセスが許可されているように、条件で問題なくなっていることを確認してください。

https://huggingface.co/google/gemma-2-2b-it

In [19]:
# Gemmaはシステム・プロンプトをサポートしていないらしい。
messages = [
    {"role": "user", "content": "Tell a light-hearted joke for a room of Data Scientists"}
  ]
generate(GEMMA2, messages)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

<bos><start_of_turn>user
Tell a light-hearted joke for a room of Data Scientists<end_of_turn>
<start_of_turn>model


The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


Why did the data scientist break up with the statistician? 

Because they had too many differences! 😂 

---

Let me know if you'd like to hear another joke! 😊 
<end_of_turn>


## エクササイズ（QWEN2、MIXTRAL）
- QWEN2 = "Qwen/Qwen2-7B-Instruct"
- MIXTRAL = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [ ]:
# ...